In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8l.pt")

results = model("simin2.jpg")

for result in results:
  result.save(filename="result.jpg")

100%|██████████| 83.7M/83.7M [00:00<00:00, 126MB/s]



image 1/1 /content/simin2.jpg: 640x640 1 person, 1 bicycle, 1 motorcycle, 1 bottle, 4512.7ms
Speed: 12.3ms preprocess, 4512.7ms inference, 44.0ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
model.export(format="onnx")

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'yolov8l.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 235.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 231.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 243.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 155.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 207.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 10.2s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.36...
ON

'yolov8l.onnx'

In [ ]:
onnx_model = YOLO("yolov8l.onnx")

results = onnx_model("simin2.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8l.onnx for ONNX Runtime inference...
Preferring ONNX Runtime AzureExecutionProvider

image 1/1 /content/simin2.jpg: 640x640 1 person, 1 bicycle, 1 motorcycle, 1 bottle, 2831.0ms
Speed: 3.2ms preprocess, 2831.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


Scikit-Learn

In [ ]:
!pip install skl2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 3.20.2 which is incompatible.


In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [ ]:
X[0]

array([5.1, 3.5, 1.4, 0.2], dtype=float32)

In [ ]:
import skl2onnx

onnx_model = skl2onnx.to_onnx(model, X[0])
with open("scikit_learn_onnx_model.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())


PyTorch

In [ ]:
!pip install onnx
!pip install onnxscript


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.8/720.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_model = torch.onnx.dynamo_export(torch_model, torch_input)

/usr/local/lib/python3.10/dist-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/local/lib/python3.10/dist-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/_exporter_legacy.py:116: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/fx/onnxfunction_dispatcher.py:503: FutureWarning: 'onnxscript.values.TracedOnnxFunction.param_schemas' is deprecated in version 0.

In [ ]:
onnx_model.save("my_image_classifier.onnx")

In [ ]:
import onnx
onnx_model = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

ModuleNotFoundError: No module named 'onnx'

TensorFlow

In [ ]:
!pip install tf2onnx

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50

model = ResNet50(weights="imagenet", include_top=True)


In [ ]:
import tf2onnx

input_data = [tf.TensorSpec(
        model.inputs[0].shape,
        dtype=model.inputs[0].dtype,
        name=model.inputs[0].name
    )]

onnx_model, _ = tf2onnx.convert.from_keras(
    model, input_data)


In [ ]:
onnx.save(onnx_model, "resnet50.onnx")

TORCH

In [9]:
!pip install torch torchvision onnx onnx-simplifier

In [14]:
import torch
import torch.onnx

# Example: Load a pre-trained model (e.g., a ResNet)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

# Create a dummy input tensor with the required input shape (e.g., batch size 1, 3 color channels, 224x224 image)
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(
    model,               # PyTorch model
    dummy_input,         # Dummy input for the model
    "model.onnx",        # Output file name
    opset_version=11,    # ONNX opset version
    input_names=['input'],  # Input tensor name
    output_names=['output'], # Output tensor name
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # Allow dynamic batch size
)

print("Model has been successfully converted to ONNX format.")


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Model has been successfully converted to ONNX format.


SCRFD

In [55]:
!pip install mmdet
!pip install mmcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.1/479.1 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv: filename=mmcv-2.2.0-cp310-cp310-linux_x86_64.whl size=28696974 sha256=5fb475a20a268e5ec5865bba2d633cb1e173e64295a66b72e6e4b282f0208ffb
  Stored in directory: /root/.cache/pip/wheels/dd/50/9f/4e0c45c952bddd25de1cae1b310bd1b20dd69967b5202ca1af
Successfully built mmcv


In [56]:
!python scrfd2onnx.py base_500m.py model.pth --output-file output.onnx --shape 640 640


Traceback (most recent call last):
  File "/content/scrfd2onnx.py", line 10, in <module>
    from mmdet.core import (build_model_from_cfg, generate_inputs_and_wrap_model,
  File "/usr/local/lib/python3.10/dist-packages/mmdet/__init__.py", line 25, in <module>
    and mmcv_version <= digit_version(mmcv_maximum_version)), \
AssertionError: MMCV==2.2.0 is used but incompatible. Please install mmcv>=1.3.17, <=1.6.0.
